# Basic implementation using jupyter notebook

In [ ]:
import logging, numpy as np
from scipy import signal
from matplotlib import pyplot as plt
import cv2
import platform

FORMAT = '[%(asctime)s] [%(levelname)s] [%(funcName)s] [%(lineno)d] : %(message)s'
logging.basicConfig(format=FORMAT, level=logging.INFO)

logging.info("Starting ...")
if platform.system() == "Windows":
    seperator = "\\"
else:
    seperator = "/"

dir = "perry-all-2"


#### Plot results:

In [ ]:
def plot_result(greens, reds, blues, x_value, title=""):
    logging.info("Plotting results ..." + title)
    x_value = x_value.reshape(1, 179)
    r = reds.tolist()[0]
    b = blues.tolist()[0]
    g = greens.tolist()[0]
    x = x_value.tolist()[0]
    plt.figure(figsize=(6, 5))
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.subplot(3, 1, 1)
    plt.plot(x, g, color="green")
    plt.subplot(3, 1, 2)
    plt.plot(x, r, color="red")
    plt.subplot(3, 1, 3)
    plt.plot(x, b, color="blue")
    plt.show()
    logging.info("Showing result")


#### rotate image:

In [ ]:
def rotate_image(mat, angle):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    """

    height, width = mat.shape[:2]  # image shape has 3 dimensions
    image_center = (
        width / 2,
        height / 2)  # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0, 0])
    abs_sin = abs(rotation_mat[0, 1])

    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w / 2 - image_center[0]
    rotation_mat[1, 2] += bound_h / 2 - image_center[1]

    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat


### Parse roi:

In [ ]:
def parse_roi(image):
    """
    Upon receiving an image, finds a face (if exists) and writes it as an image
    :param image: the image to be parsed
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # perform grayscale
    flag_face_detected = False
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    face = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(30, 30)
    )
    for (x, y, w, h) in face:
        flag_face_detected = True
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        roi_color = image[y:y + h, x:x + w]
        # print("[INFO] Object found. Saving locally.")
        cv2.imwrite('faces_detected.jpg', roi_color)
        roi_color_rgb = cv2.cvtColor(roi_color, cv2.COLOR_BGR2RGB)
        # plt.imshow(roi_color_rgb)
        # plt.show()
    if not flag_face_detected:
        logging.warning("No face detected in image")


### calculate mean:

In [ ]:
def parse_RGB(image, vidcap, greens, reds, blues, frame_number):
    """
    Parses an image to its RGB channels
    :param image: the image to be parsed
    :param vidcap:
    :param greens: array containing green channel values
    :param blues: array containing blue channel values
    :param reds: array containing red channel values
    :param frame_number - is the number of the frame of the video.
    :return: a flag indicating if there is a next image, and the next image
    """
    blue, green, red = cv2.split(image)
    greens[0, frame_number] = np.mean(green)
    blues[0, frame_number] = np.mean(blue)
    reds[0, frame_number] = np.mean(red)
    success, image = vidcap.read()
    return success, image


# %% md

### Butter filter:

# %%s
def apply_butter_filter(sig,fs):
    """
    Filter signal using Butter filter method.
    :param sig: is the signal to be filtered
    :return: the filtered signal
    """
    fc = np.array([0.1,4])
    wn =2*fc/fs
    b, a = signal.butter(4, wn, btype='bandpass')
    filt = signal.lfilter(b, a, sig)
    return filt


### read video frames:

In [ ]:
dataset_location = ".." + seperator + "dataset" + seperator + "good_sync" + seperator
specific_dir = dir
video_location = dataset_location + specific_dir + seperator + "test.mp4"
logging.info("Working on video " + video_location)
vidcap = cv2.VideoCapture(video_location)
success, image = vidcap.read()
# image = cv2.rotate(image, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
fps = vidcap.get(cv2.CAP_PROP_FPS)
number_of_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
x_value = np.arange(np.ceil(number_of_frames / fps), step=(1 / 30))
greens = np.zeros((1, int(number_of_frames)))  # instead of lists
reds = np.zeros((1, int(number_of_frames)))
blues = np.zeros((1, int(number_of_frames)))
x_value = x_value[:-1]
frame_number = 0
logging.info("Parsing images ...")
while success:
    # image = rotate_image(image, 90)
    parse_roi(image)  # build image ROI
    image = cv2.imread("faces_detected.jpg")
    success, image = parse_RGB(image, vidcap, greens, reds, blues, frame_number)
    frame_number += 1
    # image = cv2.rotate(image, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
    # cv2.imshow("Rotated (Correct)", image)
plot_result(greens, reds, blues, x_value, "All 3 channels")
greens_buttered = apply_butter_filter(greens,fs=179)
blues_buttered = apply_butter_filter(blues,fs=179)
reds_buttered = apply_butter_filter(reds,fs=179)
plt.figure(figsize=(6, 5))
plt.title("Buttered band pass")
plt.subplot(3,1,1)
plt.plot(x_value.tolist(), greens_buttered.tolist()[0], color="green")
plt.subplot(3,1,2)
plt.plot(x_value.tolist(), reds_buttered.tolist()[0], color="red")
plt.subplot(3,1,3)
plt.plot(x_value.tolist(), blues_buttered.tolist()[0], color="blue")
plt.show()
logging.info("Finished parsing the video.")

# Basic implementation using jupyter notebook

In [ ]:
import logging, numpy as np
from scipy import signal
from matplotlib import pyplot as plt
import cv2
import platform

FORMAT = '[%(asctime)s] [%(levelname)s] [%(funcName)s] [%(lineno)d] : %(message)s'
logging.basicConfig(format=FORMAT, level=logging.INFO)

logging.info("Starting ...")
if platform.system() == "Windows":
    seperator = "\\"
else:
    seperator = "/"

dir = "perry-all-2"


# %% md

#### Plot results:

# %%

def plot_result(greens, reds, blues, x_value, title=""):
    logging.info("Plotting results ..." + title)
    x_value = x_value.reshape(1, 179)
    r = reds.tolist()[0]
    b = blues.tolist()[0]
    g = greens.tolist()[0]
    x = x_value.tolist()[0]
    plt.figure(figsize=(6, 5))
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.subplot(3, 1, 1)
    plt.plot(x, g, color="green")
    plt.subplot(3, 1, 2)
    plt.plot(x, r, color="red")
    plt.subplot(3, 1, 3)
    plt.plot(x, b, color="blue")
    plt.show()
    logging.info("Showing result")


#### rotate image:

In [ ]:
def rotate_image(mat, angle):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    """

    height, width = mat.shape[:2]  # image shape has 3 dimensions
    image_center = (
        width / 2,
        height / 2)  # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0, 0])
    abs_sin = abs(rotation_mat[0, 1])

    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w / 2 - image_center[0]
    rotation_mat[1, 2] += bound_h / 2 - image_center[1]

    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat


### Parse roi:

In [ ]:
def parse_roi(image):
    """
    Upon receiving an image, finds a face (if exists) and writes it as an image
    :param image: the image to be parsed
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # perform grayscale
    flag_face_detected = False
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    face = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(30, 30)
    )
    for (x, y, w, h) in face:
        flag_face_detected = True
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        roi_color = image[y:y + h, x:x + w]
        # print("[INFO] Object found. Saving locally.")
        cv2.imwrite('faces_detected.jpg', roi_color)
        roi_color_rgb = cv2.cvtColor(roi_color, cv2.COLOR_BGR2RGB)
        # plt.imshow(roi_color_rgb)
        # plt.show()
    if not flag_face_detected:
        logging.warning("No face detected in image")


### calculate mean:

In [ ]:
def parse_RGB(image, vidcap, greens, reds, blues, frame_number):
    """
    Parses an image to its RGB channels
    :param image: the image to be parsed
    :param vidcap:
    :param greens: array containing green channel values
    :param blues: array containing blue channel values
    :param reds: array containing red channel values
    :param frame_number - is the number of the frame of the video.
    :return: a flag indicating if there is a next image, and the next image
    """
    blue, green, red = cv2.split(image)
    greens[0, frame_number] = np.mean(green)
    blues[0, frame_number] = np.mean(blue)
    reds[0, frame_number] = np.mean(red)
    success, image = vidcap.read()
    return success, image


### Butter filter:

In [ ]:
def apply_butter_filter(sig,fs):
    """
    Filter signal using Butter filter method.
    :param sig: is the signal to be filtered
    :return: the filtered signal
    """
    fc = np.array([0.1,4])
    wn =2*fc/fs
    b, a = signal.butter(4, wn, btype='bandpass')
    filt = signal.lfilter(b, a, sig)
    return filt


# %% md

### read video frames:

# %%

dataset_location = ".." + seperator + "dataset" + seperator + "good_sync" + seperator
specific_dir = dir
video_location = dataset_location + specific_dir + seperator + "test.mp4"
logging.info("Working on video " + video_location)
vidcap = cv2.VideoCapture(video_location)
success, image = vidcap.read()
# image = cv2.rotate(image, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
fps = vidcap.get(cv2.CAP_PROP_FPS)
number_of_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
x_value = np.arange(np.ceil(number_of_frames / fps), step=(1 / 30))
greens = np.zeros((1, int(number_of_frames)))  # instead of lists
reds = np.zeros((1, int(number_of_frames)))
blues = np.zeros((1, int(number_of_frames)))
x_value = x_value[:-1]
frame_number = 0
logging.info("Parsing images ...")
while success:
    # image = rotate_image(image, 90)
    parse_roi(image)  # build image ROI
    image = cv2.imread("faces_detected.jpg")
    success, image = parse_RGB(image, vidcap, greens, reds, blues, frame_number)
    frame_number += 1
    # image = cv2.rotate(image, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
    # cv2.imshow("Rotated (Correct)", image)
plot_result(greens, reds, blues, x_value, "All 3 channels")
greens_buttered = apply_butter_filter(greens,fs=179)
blues_buttered = apply_butter_filter(blues,fs=179)
reds_buttered = apply_butter_filter(reds,fs=179)
plt.figure(figsize=(6, 5))
plt.title("Buttered band pass")
plt.subplot(3,1,1)
plt.plot(x_value.tolist(), greens_buttered.tolist()[0], color="green")
plt.subplot(3,1,2)
plt.plot(x_value.tolist(), reds_buttered.tolist()[0], color="red")
plt.subplot(3,1,3)
plt.plot(x_value.tolist(), blues_buttered.tolist()[0], color="blue")
plt.show()
logging.info("Finished parsing the video.")
